# Data Pull Function

In [30]:
api_key = "" #insert your unique api key here

In [31]:
import requests
from IPython.display import display
import pickle
import io
import pandas as pd
from urllib.parse import quote


def get_data(library, 
             ticker, 
             api_key,
             csv=False, 
             save_csv=False,
             save_as_path=None,
             option_year=None):
    '''
    Function to get data from the API call
    '''
    ticker = quote(ticker)
    url = f"https://hdb.ar1th.com/historical-data?library={library}&ticker={ticker}&api_key={api_key}&option_year={option_year}"
    response = requests.get(url)
    data = io.StringIO(response.text)
    
    if library == 'Option':
        date_index_col_name = 'tx_date'
    else:
        date_index_col_name = 'Date'
        
    if csv:
        if save_csv:
            with open(f'{ticker}.csv', 'w') as f:
                f.write(response.text)
    

    df = pd.read_csv(data)
    if library != "maps":
        df.set_index(date_index_col_name, inplace=True)
        df.columns.name = ticker
        df.index.name = 'Date'
        df.index = pd.to_datetime(df.index, format='%Y-%m-%d')
        pass
        
    df.columns.name = ticker

    return df
    
def get_bulk_data(library, 
                  tickers, 
                  api_key,
                  multi_index=True):
    '''
    Function to get bulk data from the API call
    (NOT FOR LIBRARY: maps)
    '''
    if multi_index:
        data = []
        for ticker in tickers:
            temp_df = get_data(library, ticker, api_key)
            temp_df.columns = pd.MultiIndex.from_product([[ticker], temp_df.columns])
            data.append(temp_df)
            
        
        data = pd.concat(data, axis=1)
        
    else:
        data = {}
        for ticker in tickers:
            data[ticker] = get_data(library, ticker, api_key)
    
    return data

# Ways to use function

In [32]:
libraries = ["maps", "US_Equity", "World_Indices","US_Indices", "Forex", "Cash_Commodities", "Futures_Continuous", "Futures_IndividualContracts"]

# Use Maps to get a dataframe or .csv of the tickers in the library and their attributes

In [33]:
for library in libraries[1:]:
    data = get_data(library="maps", 
                    ticker=library, 
                    api_key=api_key)
    display(library)
    display(data)
    display("---"*50)

'US_Equity'

US_Equity,symbol,assetid_norgate,securityname_norgate,id Class_TRBC_1,id Class_TRBC_2,id Class_TRBC_3,id Class_TRBC_4,id Class_TRBC_5,id Class_GICS_1,id Class_GICS_2,...,name Class_GICS_4,exchange_name,exchange_name_full,base_type,subtype1,subtype2,subtype3,financial_summary,business_summary,first_quoted_date
0,A,131684,Agilent Technologies Inc Common,5.600000e+09,5.610000e+09,5.610100e+09,5.610101e+09,5.610101e+09,35000000.0,35200000.0,...,Life Sciences Tools & Services,NYSE,New York Stock Exchange,Stock Market,Equity,Operating/Holding Company,NaN,BRIEF: For the three months ended 31 January ...,"Agilent Technologies, Inc. is engaged in provi...",1999-11-18
1,AA,2214917,Alcoa Corp Common,5.100000e+09,5.120000e+09,5.120100e+09,5.120103e+09,5.120103e+09,15000000.0,15100000.0,...,Aluminum,NYSE,New York Stock Exchange,Stock Market,Equity,Operating/Holding Company,NaN,BRIEF: For the three months ended 31 March 20...,Alcoa Corporation is a vertically integrated a...,2016-10-18
2,AAA,2730624,Alternative Access First Priority CLO Bond ETF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NYSE Arca,NYSE Arca,Stock Market,Exchange Traded Product,Exchange Traded Fund (ETF),NaN,NaN,NaN,2020-09-09
3,AAAU,2425186,Goldman Sachs Physical Gold ETF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Cboe BZX,Cboe BZX,Stock Market,Exchange Traded Product,Exchange Traded Fund (ETF),NaN,NaN,Colorlight Cloud Tech Ltd is a China-based com...,2018-08-15
4,AACG,368605,ATA Creativity Global ADR,6.300000e+09,6.310000e+09,6.310100e+09,6.310101e+09,6.310101e+09,25000000.0,25300000.0,...,Education Services,Nasdaq,Nasdaq,Stock Market,Equity,Operating/Holding Company,NaN,BRIEF: For the fiscal year ended 31 December ...,ATA Creativity Global (ATA) is a China-based c...,2008-01-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12312,ZVSA,3011810,Zyversa Therapeutics Inc Common,5.600000e+09,5.620000e+09,5.620200e+09,5.620201e+09,5.620201e+09,35000000.0,35200000.0,...,Biotechnology,Nasdaq,Nasdaq,Stock Market,Equity,Operating/Holding Company,NaN,BRIEF: For the fiscal year ended 31 December ...,"ZyVersa Therapeutics, Inc. is a clinical-stage...",2022-02-10
12313,ZWS,610041,Zurn Elkay Water Solutions Corp Common,5.300000e+09,5.320000e+09,5.320300e+09,5.320302e+09,5.320302e+09,20000000.0,20100000.0,...,Building Products,NYSE,New York Stock Exchange,Stock Market,Equity,Operating/Holding Company,NaN,BRIEF: For the fiscal year ended 31 December ...,Zurn Elkay Water Solutions Corp is a pure-play...,2012-03-29
12314,ZYME,2268055,Zymeworks Inc Common,5.600000e+09,5.620000e+09,5.620200e+09,5.620201e+09,5.620201e+09,35000000.0,35200000.0,...,Biotechnology,Nasdaq,Nasdaq,Stock Market,Equity,Operating/Holding Company,NaN,BRIEF: For the fiscal year ended 31 December ...,Zymeworks Inc. is a global biotechnology compa...,2017-04-28
12315,ZYXI,126874,Zynex Inc Common,5.600000e+09,5.610000e+09,5.610100e+09,5.610101e+09,5.610101e+09,35000000.0,35100000.0,...,Health Care Equipment,Nasdaq,Nasdaq,Stock Market,Equity,Operating/Holding Company,NaN,BRIEF: For the fiscal year ended 31 December ...,"Zynex, Inc. develops, manufactures, markets an...",2002-12-05


'------------------------------------------------------------------------------------------------------------------------------------------------------'

'World_Indices'

World_Indices,index,symbol,assetid,securityname,currency,exchange_name,exchange_name_full,base_type,subtype1,subtype2,subtype3,financial_summary,business_summary,first_quoted_date,last_quoted_date,second_last_quoted_date
0,0,$BVSP,470312,Bovespa Index,USD,BOVESPA,Bolsa de Valores de Sao Paulo,Stock Market,Index,NaN,NaN,NaN,NaN,1997-03-03,NaN,NaN
1,1,$CAC,635,CAC 40 Index,EUR,EURONEXT,Euronext,Stock Market,Index,NaN,NaN,NaN,NaN,1987-12-31,NaN,NaN
2,2,$DAX,637,DAX (Deutscher Aktienindex) Index,EUR,FWB,Frankfurt Stock Exchange,Stock Market,Index,NaN,NaN,NaN,NaN,1987-12-30,NaN,NaN
3,3,$E3X,608254,FTSEurofirst 300 Index,EUR,EU IDX,Europe Non-Exchange Aligned Equity Index,Stock Market,NaN,NaN,NaN,NaN,NaN,1997-07-28,NaN,NaN
4,4,$FT100,638,FTSE 100 Index,GBP,LSE,London Stock Exchange,Stock Market,Index,NaN,NaN,NaN,NaN,1984-04-02,NaN,NaN
5,5,$FTSEMIB,608256,FTSE MIB Index,EUR,Borsa Italiana,Borsa Italiana S.P.A.,Stock Market,Index,NaN,NaN,NaN,NaN,1997-12-31,NaN,NaN
6,6,$HS,639,Hang Seng Index,HKD,SEHK,Hong Kong Stock Exchange,Stock Market,Index,NaN,NaN,NaN,NaN,1969-11-24,NaN,NaN
7,7,$IBEX,622822,IBEX 35 Index,EUR,Bolsa de Madrid,Bolsa de Madrid,Stock Market,Index,NaN,NaN,NaN,NaN,1991-09-06,NaN,NaN
8,8,$KO,640,KOSPI 200 Index,KRW,KSE,Korea Exchange,Stock Market,Index,NaN,NaN,NaN,NaN,1990-01-03,NaN,NaN
9,9,$MXX,608257,Mexico IPC Index,MXP,BMV,Bolsa Mexicana de Valores (Mexican Stock Excha...,Stock Market,Index,NaN,NaN,NaN,NaN,1987-01-02,NaN,NaN


...

Futures_Continuous,index,symbol,assetid,securityname,currency,exchange_name,exchange_name_full,base_type,subtype1,subtype2,...,last_quoted_date,second_last_quoted_date,Class,tick_size,point_value,margin,first_notice_date,lowest_ever_tick_size,session_symbol,market_symbol
0,0,&6A,471488,Australian Dollar Continuous Contract,USD,CME,Chicago Mercantile Exchange (CME Group),Futures Market,Currency,NaN,...,NaN,NaN,Currency,0.000050,100000.0,1595.0,NaN,0.000050,6A,6A
1,1,&6A_CCB,686982,Australian Dollar Continuous Contract Backadju...,USD,CME,Chicago Mercantile Exchange (CME Group),Futures Market,Currency,NaN,...,NaN,NaN,Currency,0.000050,100000.0,1595.0,NaN,0.000050,6A,6A
2,2,&6B,471490,British Pound Continuous Contract,USD,CME,Chicago Mercantile Exchange (CME Group),Futures Market,Currency,NaN,...,NaN,NaN,Currency,0.000100,62500.0,2090.0,NaN,0.000100,6B,6B
3,3,&6B_CCB,686984,British Pound Continuous Contract Backadjusted,USD,CME,Chicago Mercantile Exchange (CME Group),Futures Market,Currency,NaN,...,NaN,NaN,Currency,0.000100,62500.0,2090.0,NaN,0.000100,6B,6B
4,4,&6C,471493,Canadian Dollar Continuous Contract,USD,CME,Chicago Mercantile Exchange (CME Group),Futures Market,Currency,NaN,...,NaN,NaN,Currency,0.000050,100000.0,1100.0,NaN,0.000050,6C,6C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,219,&ZS_CCB,687051,Soybean Continuous Contract Backadjusted,USD,CBOT,Chicago Board of Trade (CME Group),Futures Market,Agriculture,NaN,...,NaN,NaN,Agriculture & Livestock,0.250000,50.0,2640.0,NaN,0.250000,ZS,ZS
220,220,&ZT,471480,2-Year U.S. T-Note Continuous Contract,USD,CBOT,Chicago Board of Trade (CME Group),Futures Market,Interest Rate,NaN,...,NaN,NaN,Interest Rate,0.003906,2000.0,1320.0,NaN,0.003906,ZT,ZT
221,221,&ZT_CCB,686975,2-Year U.S. T-Note Continuous Contract Backadj...,USD,CBOT,Chicago Board of Trade (CME Group),Futures Market,Interest Rate,NaN,...,NaN,NaN,Interest Rate,0.003906,2000.0,1320.0,NaN,0.003906,ZT,ZT
222,222,&ZW,471568,Chicago SRW Wheat Continuous Contract,USD,CBOT,Chicago Board of Trade (CME Group),Futures Market,Agriculture,NaN,...,NaN,NaN,Agriculture & Livestock,0.250000,50.0,2200.0,NaN,0.250000,ZW,ZW


'------------------------------------------------------------------------------------------------------------------------------------------------------'

'Futures_IndividualContracts'

Futures_IndividualContracts,"{""message"": ""Severe Error retreiving from Database: E_ASSERTION_FAILURE Empty symbol in reference key""}"


'------------------------------------------------------------------------------------------------------------------------------------------------------'

# Data Snippet Check (Except for Individual Futures Map and Equities_options, all lib maps are available for data pull)

In [34]:
libs = libraries[1:]
libs.remove("Futures_IndividualContracts")

for lib in libs:
    display(f"Library Snipped (Displaying Head=3): {lib}")
    map_futures = get_data(library="maps",
                            ticker=lib,
                            api_key=api_key)
    map_futures = map_futures.head(3)
    display(map_futures)

    for (idx , row) in map_futures.iterrows():
        df = get_data(library=lib, 
                            ticker=row.symbol, 
                            api_key=api_key)
        
        display(df)
        
    display()
    display("---"*250)



'Library Snipped (Displaying Head=3): US_Equity'

US_Equity,symbol,assetid_norgate,securityname_norgate,id Class_TRBC_1,id Class_TRBC_2,id Class_TRBC_3,id Class_TRBC_4,id Class_TRBC_5,id Class_GICS_1,id Class_GICS_2,...,name Class_GICS_4,exchange_name,exchange_name_full,base_type,subtype1,subtype2,subtype3,financial_summary,business_summary,first_quoted_date
0,A,131684,Agilent Technologies Inc Common,5.600000e+09,5.610000e+09,5.610100e+09,5.610101e+09,5.610101e+09,35000000.0,35200000.0,...,Life Sciences Tools & Services,NYSE,New York Stock Exchange,Stock Market,Equity,Operating/Holding Company,NaN,BRIEF: For the three months ended 31 January ...,"Agilent Technologies, Inc. is engaged in provi...",1999-11-18
1,AA,2214917,Alcoa Corp Common,5.100000e+09,5.120000e+09,5.120100e+09,5.120103e+09,5.120103e+09,15000000.0,15100000.0,...,Aluminum,NYSE,New York Stock Exchange,Stock Market,Equity,Operating/Holding Company,NaN,BRIEF: For the three months ended 31 March 20...,Alcoa Corporation is a vertically integrated a...,2016-10-18
2,AAA,2730624,Alternative Access First Priority CLO Bond ETF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NYSE Arca,NYSE Arca,Stock Market,Exchange Traded Product,Exchange Traded Fund (ETF),NaN,NaN,NaN,2020-09-09


A,Open,High,Low,Close,Volume,Turnover,Unadjusted Close,Dividend,Constituent_S&P 500,Constituent_S&P MidCap 400,Constituent_S&P SmallCap 600
Date,,,,,,,,,,,
1999-11-18,27.597343,30.326752,24.261400,25.929373,73752700.0,1.968556e+09,42.7500,0.0,0,0,0
1999-11-19,26.043097,26.081005,24.147675,24.716303,17965822.0,4.399704e+08,40.7500,0.0,0,0,0
1999-11-22,25.057478,26.460090,24.299309,26.460090,7757342.5,1.943159e+08,43.6250,0.0,0,0,0
1999-11-23,25.777739,26.460090,24.413034,24.413034,7047078.5,1.768620e+08,40.2500,0.0,0,0,0
1999-11-24,24.337217,25.436562,24.261400,24.905844,5711789.0,1.422569e+08,41.0625,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
2024-02-12,133.280000,133.280000,130.380000,132.310000,1231836.0,1.625114e+08,132.3100,0.0,1,0,0
2024-02-13,130.340000,131.640000,128.260000,129.760000,1031115.0,1.336706e+08,129.7600,0.0,1,0,0
2024-02-14,131.000000,132.330000,130.209400,132.240000,1405219.0,1.848261e+08,132.2400,0.0,1,0,0


AA,Open,High,Low,Close,Volume,Turnover,Unadjusted Close,Dividend,Constituent_S&P 500,Constituent_S&P MidCap 400,Constituent_S&P SmallCap 600
Date,,,,,,,,,,,
2016-10-18,22.023520,22.023520,22.023520,22.023520,1.021635e+03,22500.0,22.50,0.0,0,0,0
2016-10-20,22.023520,23.736460,19.703707,20.555285,6.782635e+03,142189.0,21.00,0.0,0,0,0
2016-10-21,20.751049,20.751049,20.447613,20.555285,1.120019e+04,231354.0,21.00,0.0,0,0,0
2016-10-24,20.555285,20.555285,20.535707,20.555285,6.058296e+03,124520.0,21.00,0.0,0,0,0
2016-10-25,21.690720,22.023520,20.202908,20.799990,1.687302e+05,3569175.0,21.25,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
2024-02-12,27.100000,28.095000,26.870000,27.900000,4.374586e+06,121476570.0,27.90,0.0,0,1,0
2024-02-13,26.900000,27.200000,25.575000,25.850000,7.264231e+06,189461310.0,25.85,0.0,0,1,0
2024-02-14,26.210000,26.990000,25.870000,26.790000,5.100599e+06,135514240.0,26.79,0.0,0,1,0


AAA,Open,High,Low,Close,Volume,Turnover,Unadjusted Close,Dividend,Constituent_S&P 500,Constituent_S&P MidCap 400,Constituent_S&P SmallCap 600
Date,,,,,,,,,,,
2020-09-09,22.530489,22.547543,22.503470,22.503470,19303.074,434875.00,25.0699,0.0,0,0,0
2020-09-10,22.494583,22.503560,22.482016,22.501764,26163.370,588649.00,25.0680,0.0,0,0,0
2020-09-11,22.476631,22.485607,22.458680,22.472143,37166.800,835391.00,25.0350,0.0,0,0,0
2020-09-14,22.449701,22.494583,22.449701,22.458680,14645.248,329249.00,25.0200,0.0,0,0,0
2020-09-15,22.458680,22.467655,22.449701,22.449701,13445.421,301929.00,25.0100,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
2024-02-12,25.110000,25.190000,25.050000,25.120000,3870.000,97228.72,25.1200,0.0,0,0,0
2024-02-13,25.200000,25.200000,25.070000,25.135000,11079.000,278608.03,25.1350,0.0,0,0,0
2024-02-14,25.050000,25.192100,25.050000,25.135000,12152.000,304913.12,25.1350,0.0,0,0,0


...

Futures_Continuous,index,symbol,assetid,securityname,currency,exchange_name,exchange_name_full,base_type,subtype1,subtype2,...,last_quoted_date,second_last_quoted_date,Class,tick_size,point_value,margin,first_notice_date,lowest_ever_tick_size,session_symbol,market_symbol
0,0,&6A,471488,Australian Dollar Continuous Contract,USD,CME,Chicago Mercantile Exchange (CME Group),Futures Market,Currency,NaN,...,NaN,NaN,Currency,0.00005,100000.0,1595.0,NaN,0.00005,6A,6A
1,1,&6A_CCB,686982,Australian Dollar Continuous Contract Backadju...,USD,CME,Chicago Mercantile Exchange (CME Group),Futures Market,Currency,NaN,...,NaN,NaN,Currency,0.00005,100000.0,1595.0,NaN,0.00005,6A,6A
2,2,&6B,471490,British Pound Continuous Contract,USD,CME,Chicago Mercantile Exchange (CME Group),Futures Market,Currency,NaN,...,NaN,NaN,Currency,0.00010,62500.0,2090.0,NaN,0.00010,6B,6B


%266A,Open,High,Low,Close,Volume,Delivery Month,Open Interest
Date,,,,,,,
1987-01-13,0.6460,0.64700,0.64200,0.64220,1854.0,198703.0,915.0
1987-01-14,0.6321,0.63330,0.62880,0.62990,1522.0,198703.0,989.0
1987-01-15,0.6400,0.64010,0.63870,0.63890,571.0,198703.0,862.0
1987-01-16,0.6408,0.64200,0.64080,0.64170,556.0,198703.0,1040.0
1987-01-19,0.6416,0.64500,0.64160,0.64350,516.0,198703.0,943.0
...,...,...,...,...,...,...,...
2024-05-13,0.6611,0.66360,0.65930,0.66160,75250.0,202406.0,202412.0
2024-05-14,0.6615,0.66345,0.65860,0.66305,98650.0,202406.0,201697.0
2024-05-15,0.6633,0.67050,0.66280,0.66965,145548.0,202406.0,199548.0


%266A_CCB,Open,High,Low,Close,Volume,Delivery Month,Open Interest
Date,,,,,,,
1987-01-13,0.0437,0.04470,0.03970,0.03990,1854.0,198703.0,915.0
1987-01-14,0.0298,0.03100,0.02650,0.02760,1522.0,198703.0,989.0
1987-01-15,0.0377,0.03780,0.03640,0.03660,571.0,198703.0,862.0
1987-01-16,0.0385,0.03970,0.03850,0.03940,556.0,198703.0,1040.0
1987-01-19,0.0393,0.04270,0.03930,0.04120,516.0,198703.0,943.0
...,...,...,...,...,...,...,...
2024-05-13,0.6611,0.66360,0.65930,0.66160,75250.0,202406.0,202412.0
2024-05-14,0.6615,0.66345,0.65860,0.66305,98650.0,202406.0,201697.0
2024-05-15,0.6633,0.67050,0.66280,0.66965,145548.0,202406.0,199548.0


%266B,Open,High,Low,Close,Volume,Delivery Month,Open Interest
Date,,,,,,,
1979-03-20,2.0230,2.0230,2.0230,2.0230,0.0,198003.0,211.0
1979-03-21,2.0230,2.0230,2.0230,2.0230,8.0,198003.0,210.0
1979-03-22,2.0230,2.0230,2.0150,2.0230,1.0,198003.0,210.0
1979-03-23,2.0200,2.0200,2.0200,2.0200,2.0,198003.0,211.0
1979-03-26,2.0205,2.0300,2.0205,2.0300,1.0,198003.0,209.0
...,...,...,...,...,...,...,...
2024-05-13,1.2521,1.2571,1.2519,1.2559,65295.0,202406.0,218092.0
2024-05-14,1.2561,1.2595,1.2511,1.2592,113174.0,202406.0,222483.0
2024-05-15,1.2594,1.2689,1.2586,1.2683,132560.0,202406.0,218335.0


'------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------'

In [35]:
data = get_data(library="US_Equity",
         ticker="AAPL",
         api_key=api_key,
         csv=False,
         save_csv=False,
         save_as_path=None,
         )
data

AAPL,Open,High,Low,Close,Volume,Turnover,Unadjusted Close,Dividend,Constituent_S&P 500,Constituent_S&P MidCap 400,Constituent_S&P SmallCap 600
Date,,,,,,,,,,,
1980-12-12,0.099189,0.099624,0.099189,0.099189,606916800.0,6.019962e+07,28.750,0.0,0,0,0
1980-12-15,0.094014,0.094449,0.094014,0.094014,227590180.0,2.139670e+07,27.250,0.0,0,0,0
1980-12-16,0.087114,0.087549,0.087114,0.087114,136809180.0,1.191800e+07,25.250,0.0,0,0,0
1980-12-17,0.089274,0.089702,0.089274,0.089274,111853096.0,9.985548e+06,25.876,0.0,0,0,0
1980-12-18,0.091861,0.092289,0.091861,0.091861,95041800.0,8.730665e+06,26.626,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
2024-02-12,188.415000,188.670000,186.790000,187.150000,41781936.0,7.838709e+09,187.150,0.0,1,0,0
2024-02-13,185.770000,186.210000,183.512800,185.040000,56529530.0,1.045606e+10,185.040,0.0,1,0,0
2024-02-14,185.320000,185.530000,182.440000,184.150000,54630516.0,1.002769e+10,184.150,0.0,1,0,0


In [36]:

for ticker in ['AAPL', 'GOOGL']:
    data = get_data(library="US_Equity",
         ticker=ticker,
         api_key=api_key,
         csv=False,
         save_csv=False,
         save_as_path=None,
         )
    
    display(data)
    
    

AAPL,Open,High,Low,Close,Volume,Turnover,Unadjusted Close,Dividend,Constituent_S&P 500,Constituent_S&P MidCap 400,Constituent_S&P SmallCap 600
Date,,,,,,,,,,,
1980-12-12,0.099189,0.099624,0.099189,0.099189,606916800.0,6.019962e+07,28.750,0.0,0,0,0
1980-12-15,0.094014,0.094449,0.094014,0.094014,227590180.0,2.139670e+07,27.250,0.0,0,0,0
1980-12-16,0.087114,0.087549,0.087114,0.087114,136809180.0,1.191800e+07,25.250,0.0,0,0,0
1980-12-17,0.089274,0.089702,0.089274,0.089274,111853096.0,9.985548e+06,25.876,0.0,0,0,0
1980-12-18,0.091861,0.092289,0.091861,0.091861,95041800.0,8.730665e+06,26.626,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
2024-02-12,188.415000,188.670000,186.790000,187.150000,41781936.0,7.838709e+09,187.150,0.0,1,0,0
2024-02-13,185.770000,186.210000,183.512800,185.040000,56529530.0,1.045606e+10,185.040,0.0,1,0,0
2024-02-14,185.320000,185.530000,182.440000,184.150000,54630516.0,1.002769e+10,184.150,0.0,1,0,0


GOOGL,Open,High,Low,Close,Volume,Turnover,Unadjusted Close,Dividend,Constituent_S&P 500,Constituent_S&P MidCap 400,Constituent_S&P SmallCap 600
Date,,,,,,,,,,,
2004-08-19,2.502673,2.604021,2.401325,2.510681,893422140.0,2.242790e+09,100.33,0.0,0,0,0
2004-08-20,2.539459,2.729643,2.514935,2.710374,457376740.0,1.237832e+09,108.31,0.0,0,0,0
2004-08-23,2.773685,2.839749,2.728892,2.737651,365442660.0,9.996097e+08,109.40,0.0,0,0,0
2004-08-24,2.785197,2.792704,2.591759,2.624291,305544700.0,8.002944e+08,104.87,0.0,0,0,0
2004-08-25,2.626543,2.702617,2.599517,2.652568,183978110.0,4.874834e+08,106.00,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
2024-02-12,148.420000,149.335000,147.370000,147.530000,21564084.0,3.194907e+09,147.53,0.0,1,0,0
2024-02-13,144.920000,146.670000,143.690000,145.140000,27837730.0,4.037771e+09,145.14,0.0,1,0,0
2024-02-14,146.080000,146.520000,144.094000,145.940000,22704166.0,3.299653e+09,145.94,0.0,1,0,0


In [37]:
df = get_bulk_data(library="US_Equity",
                    tickers=['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA'],
                    api_key=api_key,
                    multi_index=True)
display(df)

AAPL                                                   \
                  Open        High         Low       Close       Volume   
Date                                                                      
1980-12-12    0.099189    0.099624    0.099189    0.099189  606916800.0   
1980-12-15    0.094014    0.094449    0.094014    0.094014  227590180.0   
1980-12-16    0.087114    0.087549    0.087114    0.087114  136809180.0   
1980-12-17    0.089274    0.089702    0.089274    0.089274  111853096.0   
1980-12-18    0.091861    0.092289    0.091861    0.091861   95041800.0   
...                ...         ...         ...         ...          ...   
2024-02-12  188.415000  188.670000  186.790000  187.150000   41781936.0   
2024-02-13  185.770000  186.210000  183.512800  185.040000   56529530.0   
2024-02-14  185.320000  185.530000  182.440000  184.150000   54630516.0   
2024-02-15  183.550000  184.490000  181.350000  183.860000   65434496.0   
2024-02-16  183.420000  184.850000  181.665000  182.310000   49752464.0   

                                                                        \
                Turnover Unadjusted Close Dividend Constituent_S&P 500   
Date                                                                     
1980-12-12  6.019962e+07           28.750      0.0                   0   
1980-12-15  2.139670e+07           27.250      0.0                   0   
1980-12-16  1.191800e+07           25.250      0.0                   0   
1980-12-17  9.985548e+06           25.876      0.0                   0   
1980-12-18  8.730665e+06           26.626      0.0                   0   
...                  ...              ...      ...                 ...   
2024-02-12  7.838709e+09          187.150      0.0                   1   
2024-02-13  1.045606e+10          185.040      0.0                   1   
2024-02-14  1.002769e+10          184.150      0.0                   1   
2024-02-15  1.195136e+10          183.860      0.0                   1   
2024-02-16  9.092806e+09          182.310      0.0                   1   

                                       ...    TSLA                    \
           Constituent_S&P MidCap 400  ...    High       Low   Close   
Date                                   ...                             
1980-12-12                          0  ...     NaN       NaN     NaN   
1980-12-15                          0  ...     NaN       NaN     NaN   
1980-12-16                          0  ...     NaN       NaN     NaN   
1980-12-17                          0  ...     NaN       NaN     NaN   
1980-12-18                          0  ...     NaN       NaN     NaN   
...                               ...  ...     ...       ...     ...   
2024-02-12                          0  ...  194.73  187.2800  188.13   
2024-02-13                          0  ...  187.26  182.1087  184.02   
2024-02-14                          0  ...  188.89  183.3500  188.71   
2024-02-15                          0  ...  200.88  188.8595  200.45   
2024-02-16                          0  ...  203.17  197.4000  199.95   

                                                                 \
                 Volume      Turnover Unadjusted Close Dividend   
Date                                                              
1980-12-12          NaN           NaN              NaN      NaN   
1980-12-15          NaN           NaN              NaN      NaN   
1980-12-16          NaN           NaN              NaN      NaN   
1980-12-17          NaN           NaN              NaN      NaN   
1980-12-18          NaN           NaN              NaN      NaN   
...                 ...           ...              ...      ...   
2024-02-12   95498600.0  1.819555e+10           188.13      0.0   
2024-02-13   86759480.0  1.598062e+10           184.02      0.0   
2024-02-14   81202984.0  1.511240e+10           188.71      0.0   
2024-02-15  120831760.0  2.367592e+10           200.45      0.0   
2024-02-16  111346700.0  2.229546e+10           19

In [38]:
dict_data = get_bulk_data(library="US_Equity",
                    tickers=['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA'],
                    api_key=api_key,
                    multi_index=False)
display(dict_data)

{'AAPL': AAPL              Open        High         Low       Close       Volume  \
 Date                                                                      
 1980-12-12    0.099189    0.099624    0.099189    0.099189  606916800.0   
 1980-12-15    0.094014    0.094449    0.094014    0.094014  227590180.0   
 1980-12-16    0.087114    0.087549    0.087114    0.087114  136809180.0   
 1980-12-17    0.089274    0.089702    0.089274    0.089274  111853096.0   
 1980-12-18    0.091861    0.092289    0.091861    0.091861   95041800.0   
 ...                ...         ...         ...         ...          ...   
 2024-02-12  188.415000  188.670000  186.790000  187.150000   41781936.0   
 2024-02-13  185.770000  186.210000  183.512800  185.040000   56529530.0   
 2024-02-14  185.320000  185.530000  182.440000  184.150000   54630516.0   
 2024-02-15  183.550000  184.490000  181.350000  183.860000   65434496.0   
 2024-02-16  183.420000  184.850000  181.665000  182.310000   49752464.0   
 
 A